In [1]:
#
# Revisar manualmente as imagens em C:\mestrado\datasets\samples_font  ok
#
#
# Pegar todas as imagens em final-32K-8lines do tipo sync_**.jpg e sobrescrever
# usando como texto a sequencia no id correspondente em portugues.
#
# Para os fontes carregar a lista de fontes cujas imagens existem em samples_font
# buscando as fontes em C:/fonts, mais as fontes do windows usado anteriormente
# Usar as fontes aleatoriamente...
#


In [2]:
print( 11)

11


In [2]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
import random
import pickle
import cv2
import os.path
from os import path
import shutil
from glob import glob
from shutil import copyfile
from pathlib import Path
from matplotlib import pyplot as plt
from IPython.display import clear_output, display
from collections import Counter
import builtins

from sklearn.utils import shuffle

#
# gera imagem com jogadas escritas com fonte manuscrito
#
# -*- coding: utf-8 -*-
import PIL
from PIL import ImageFont
from PIL import Image, ImageDraw
from PIL import ImageColor

from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

# print(tf.__version__) 

In [3]:
def read_label( path):
  f= open( path)
  d= f.read()
  f.close()
  return d 

def write_label( path, text):
  f= open( path, "w")
  f.write( text)
  f.close()

def show( img, name= 'noname'):
  plt.imshow( img)
  plt.title( name)
  plt.show()

def show_gimage( img, name= 'noname'):
  plt.imshow( img, cmap= 'gray')
  plt.title( name)
  plt.show()

def _load_image( path):
  # return cv2.imread( path, cv2.IMREAD_GRAYSCALE)
  return cv2.imread( path)
  
def write_image( path, img):
  cv2.imwrite( path, img)

def resize( img, scale_percent= 60, verbose=False):
  #scale_percent = 60 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  if verbose:
    print( "scale", scale_percent)
    print( "dim", dim)
  # resize image
  return cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

In [15]:

# https://gist.github.com/erniejunior/601cdf56d2b424757de5
def elastic_transform_2(image, alpha, sigma,random_state=None):
#     print( alpha, sigma)
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dz = np.zeros_like(dx)

#     x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
#     print (x.shape)
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    distored_image = map_coordinates(image, indices, order=1, mode='reflect')
    return distored_image.reshape(image.shape)

In [16]:
def transform( img):
    # ELASTIC_ALPHA_SIGMA = ((1201, 10), (1501, 12), (991, 8))
    if type( img)== PIL.Image.Image:
        img= np.asarray( img)
        img= elastic_transform_2(img, 1201, 10)  #random.randint( 991, 1501), random.randint( 8, 12))
        img= PIL.Image.fromarray( img)
        return img
    else:
        return elastic_transform_2(img, 1201, 10)  #random.randint( 991, 1501), random.randint( 8, 12))
    

In [10]:
# pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_pt.txt"
# with open(pgnfile) as f:
#     lines = [line.rstrip() for line in f]
    
# lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
# lines= [ln[8:] for ln in lines]
# lines= [ln.split()[0:32] for ln in lines]

# pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_eng.txt"
# with open(pgnfile) as f:
#     en_lines = [line.rstrip() for line in f]
    
# en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
# en_lines= [ln[8:] for ln in en_lines]
# en_lines= [ln.split()[0:32] for ln in en_lines]

# pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_pt.txt"
# with open(pgnfile) as f:
#     lines = [line.rstrip() for line in f]
    
# lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
# lines= [ln[8:] for ln in lines]
# lines= [ln.split() for ln in lines]

# en_pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_eng.txt"
# with open(en_pgnfile) as f:
#     en_lines = [line.rstrip() for line in f]
    
# en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
# en_lines= [ln[8:] for ln in en_lines]
# en_lines= [ln.split() for ln in en_lines]

In [25]:
# print( lines[0], en_lines[0])

['d4', 'f5', 'Cf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Cf6', 'b4', 'c6', 'c4', 'd5', 'Db3', 'Be6', 'c5', 'Ce4', 'O-O', 'Cd7', 'Bf4', 'O-O', 'Cbd2', 'Bf7', 'De3', 'De8', 'Bh6', 'Cdf6', 'Ce5', 'Cxd2', 'Dxd2', 'Cd7', 'Bxg7', 'Rxg7', 'Df4', 'Be6', 'a4', 'Cxe5', 'Dxe5+', 'Tf6', 'f4', 'Rg8', 'Tfb1', 'Bc8', 'Dc7', 'Dd7', 'Dxd7', 'Bxd7', 'b5', 'Tff8', 'e3', 'e6', 'Bf1', 'Rf7', 'a5', 'Tfb8', 'Tb2', 'Re7', 'Tab1', 'Tc8', 'bxc6', 'bxc6', 'Ba6', 'Tc7', 'Tb8', 'Txb8', 'Txb8', 'Be8', 'Bb7', 'Bd7', 'a6', 'Be8', 'Ta8', 'Bd7', 'Rf2', 'Rf6', 'Th8', 'Rg7', 'Ta8', 'Rf6', 'Re2', 'h6', 'Rd2', 'g5', 'Rc3', 'gxf4', 'exf4'] ['d4', 'f5', 'Nf3', 'g6', 'g3', 'Bg7', 'Bg2', 'Nf6', 'b4', 'c6', 'c4', 'd5', 'Qb3', 'Be6', 'c5', 'Ne4', 'O-O', 'Nd7', 'Bf4', 'O-O', 'Nbd2', 'Bf7', 'Qe3', 'Qe8', 'Bh6', 'Ndf6', 'Ne5', 'Nxd2', 'Qxd2', 'Nd7', 'Bxg7', 'Kxg7', 'Qf4', 'Be6', 'a4', 'Nxe5', 'Qxe5+', 'Rf6', 'f4', 'Kg8', 'Rfb1', 'Bc8', 'Qc7', 'Qd7', 'Qxd7', 'Bxd7', 'b5', 'Rff8', 'e3', 'e6', 'Bf1', 'Kf7', 'a5', 'Rfb8', 'Rb2', 'Ke7', 'Rab1', '

In [26]:
# print( len( lines), len(en_lines))

30000 30000


In [ ]:
#
# le todos os fontes
#

In [17]:
new_fonts_files= glob( 'C:/mestrado/datasets/samples_font/*.ttf.jpg')

In [18]:
print( len( new_fonts_files))

77


In [19]:
new_fonts_path= [
    os.path.join( 'C:/mestrado/datasets/fonts', 
                 Path( f).name.replace( '.jpg', '')) for f in new_fonts_files]

In [20]:
new_fonts= [ImageFont.truetype(f, 30) for f in new_fonts_path]

In [21]:
fonts= [
    ImageFont.truetype('c:/Windows/Fonts/BRADHITC.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/ITCKRIST.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/LHANDW.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/Inkfree.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/OCRAEXT.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/TEMPSITC.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/PAPYRUS.TTF', 30),
    ImageFont.truetype('c:/Windows/Fonts/MAIAN.TTF', 30)
] 

In [22]:
all_fonts= new_fonts + fonts

In [23]:
print( len( new_fonts), len( fonts), len( all_fonts))

77 8 85


In [24]:
def showbig( img, name= 'noname'):
    plt.figure(figsize = (10,10))    
    plt.imshow( img)
    plt.title( name)
    plt.show()

In [47]:
def sp_noise(npimage,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(npimage.shape,np.uint8)
    thres = 1 - prob 
    for i in range(npimage.shape[0]):
        for j in range(npimage.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = random.randint( 0, 128)
            else:
                output[i][j] = npimage[i][j]
    return output

def sp_noise_pil( pilimg, prob):
#     print( 'noise', prob)
    img= pilimg
    img= np.asarray( img)
    img= sp_noise(img, prob)
    img= PIL.Image.fromarray( img)
    return img

def distorce( img):
    # ELASTIC_ALPHA_SIGMA = ((1201, 10), (1501, 12), (991, 8))
#     print( 'distorce')
    if type( img)== PIL.Image.Image:
        img= np.asarray( img)
        img= elastic_transform_2(img, 991, 8)  #random.randint( 991, 1501), random.randint( 8, 12))
        img= PIL.Image.fromarray( img)
        return img
    else:
        return elastic_transform_2(img, 991, 8)  #random.randint( 991, 1501), random.randint( 8, 12))
    

# Rotate the foreground
def rotate( pilimg):
#     print( 'rotate')
    foreground= pilimg
    angle_degrees = random.uniform(-2.5,2.5)
#     print( angle_degrees)
    foreground = foreground.rotate(angle_degrees, resample=Image.BICUBIC, expand=True, fillcolor = (255,255,255))

    # Scale the foreground
#     scale = random.random() * .5 + .5 # Pick something between .5 and 1
#     scale = 1
#     new_size = (int(foreground.size[0] * scale), int(foreground.size[1] * scale))
#     foreground = foreground.resize(new_size, resample=Image.BICUBIC)
    return foreground
#     show( foreground)
    

def random_transform( pilimg, p):
    img= pilimg
    if( random.random()< 0.8):
        img= rotate( img)
#     if( random.random()< 0.1):
#         img= distorce( img)
    if( random.random()< 0.25):
        img= sp_noise_pil( img, random.uniform( 0.001, 0.0001))
    
    return img


In [26]:
pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v5/*/labels/*.pgn')
pgn_files= [f for f in pgn_files if 'scan' in f]
print( len( pgn_files))

all_pgn= []
for f in pgn_files:
    all_pgn.append( read_label( f))
	
all_pgn= [p.split()[0:16] for p in all_pgn]
flat_all_pgn = [t for _list in all_pgn for t in _list]

count16= Counter( flat_all_pgn)

print( 'len( count16)', len( _count16))  #esperado 462
counted16= {k: v for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)}
print( counted16)



3186
len( count16) 462
{'O-O': 2690, 'Nf3': 2630, 'Nf6': 2601, 'd4': 2562, 'e4': 2135, 'Nc3': 2081, 'd5': 1719, 'e6': 1567, 'Nc6': 1545, 'c5': 1510, 'd6': 1418, 'c4': 1411, 'e5': 1338, 'g6': 1037, 'Bg7': 990, 'Be7': 797, 'cxd4': 791, 'c6': 735, 'g3': 683, 'Bg2': 650, 'a6': 641, 'e3': 639, 'Nxd4': 565, 'Bd3': 554, 'c3': 512, 'Bg5': 496, 'Be2': 475, 'd3': 447, 'exd5': 435, 'Nbd7': 420, 'Be3': 391, 'Bc4': 371, 'cxd5': 354, 'b6': 309, 'b5': 300, 'Bg4': 291, 'f4': 281, 'h3': 279, 'Bb4': 269, 'h6': 258, 'a3': 249, 'Bb5': 249, 'Bb7': 243, 'Nbd2': 239, 'b3': 209, 'Qc7': 205, 'Bf4': 200, 'Bf5': 185, 'Bd7': 185, 'Nxe4': 182, 'Nd7': 179, 'Bd6': 177, 'Qd2': 174, 'Re1': 166, 'Qc2': 165, 'f5': 165, 'exd4': 164, 'Nxd5': 161, 'dxe5': 161, 'f3': 156, 'Bb3': 151, 'Bc5': 150, 'Nd2': 149, 'Nge2': 146, 'Bb2': 144, 'a4': 143, 'dxc4': 142, 'dxe4': 140, 'Ne7': 137, 'Qb6': 131, 'Be6': 126, 'Nge7': 120, 'Nd5': 115, 'Qe2': 109, 'dxc5': 109, 'Ne4': 106, 'Ne2': 106, 'Bd2': 106, 'bxc3': 105, 'Nb3': 104, 'Qa5': 100,

In [52]:
len( all_pgn)

3186

In [27]:

# 
# transforma de portugues para ingles, para gerar pgn

def pt_to_pgn( w):
    # R=> K
    # T=> R
    # C=> N
    # B=> B
    # D=> Q

#     print( w, '=> ')
    png= w.replace( 'R', 'K').replace( 'T', 'R').replace( 'C', 'N').replace( 'D', 'Q')
#     print( png)
    return png

def to_pgn( label):
    return [pt_to_pgn( w) for w in label]


In [37]:
def pgn_to_pt( w):
    # R => T
    # K => R
    # N => C
    # Q => D
    
    return w.replace( 'R', 'T').replace( 'K', 'R').replace( 'N', 'C').replace( 'Q', 'D')


In [28]:
words= [item for item in count16]

In [29]:
print( len( words), words)

462 ['a3', 'd5', 'd4', 'Nf6', 'Nf3', 'Bg4', 'e3', 'e6', 'Be2', 'Bd6', 'c4', 'c6', 'Nbd2', 'Nbd7', 'b4', 'O-O', 'Nc3', 'Be7', 'Qc2', 'Re8', 'e5', 'e4', 'Nce2', 'c5', 'Ng3', 'Be6', 'Bb5+', 'Nd7', 'd3', 'f6', 'b3', 'Bb2', 'Bf5', 'f4', 'h6', 'Ne5', 'h5', 'g6', 'Bg7', 'Ne4', 'Nc6', 'd6', 'Bxf3', 'g3', 'Bg2', 'Ne2', 'Nge7', 'exd4', 'Bb5', 'Bd7', 'a6', 'Ba4', 'Nd4', 'Bc5', 'Nxc6', 'dxc6', 'Ngf6', 'b5', 'a4', 'dxc4', 'Bxc4', 'Nb6', 'Qc7', 'Qd6', 'cxd5', 'Nxd5', 'b6', 'Bb7', 'h3', 'c3', 'Nd2', 'Ngf3', 'Bd3', 'cxd4', 'Qe2', 'Bb4', 'f5', 'fxe4', 'Bxe4', 'Qh4', 'Rb1', 'Rb8', 'cxb4', 'axb4', 'Nxd4', 'Nb5', 'Nc7', 'Nge2', 'Bh5', 'Ne8', 'Nb8', 'bxc6', 'Bxc3', 'Na6', 'Bf4', 'Nxc3', 'bxc3', 'Bg5', 'exd5', 'Qb3', 'Nh6', 'Nce7', 'h4', 'Qd7', 'g4', 'dxe5', 'Qa5+', 'Qxe5', 'Qa5', 'Ne7', 'Nd5', 'Bxh7+', 'Kh8', 'Bf8', 'Bxc3+', 'a5', 'Qxd4', 'Nh3', 'Nf4', 'Ne1', 'Nc2', 'Nb4', 'Qc8', 'Nh4', 'Ng2', 'Nf5', 'Bxe2', 'f3', 'exf3', 'fxg2', 'cxd7+', 'Bxd7', 'Bxg2', 'Qf5', 'Nxf6+', 'gxf6', 'Nbc6', 'Nec3', 'Qa4+', 'Qxc

In [42]:
def random_label_from_vocab():
    label= []
    for i in range( 0, 16):
        label.append( 
            pgn_to_pt( words[random.randrange( 0, len( words))])
        )
    return label


print( random_label_from_vocab())


['Cbd7', 'Bh5', 'Cxb5', 'Ch6', 'Rg8', 'Bg2', 'Ce1', 'cxb4', 'Bb6', 'Txd8', 'Bf2', 'Bxb4+', 'Cgxe7', 'fxg7', 'De4', 'Cxc6']


In [31]:
dd= {w:0 for w in words}

In [32]:
for i in range( 0, 20000*16):
    w= words[random.randrange( 0, len( words))]
    dd[ w]= dd[ w]+1
    

In [33]:
print( dd)

{'a3': 621, 'd5': 722, 'd4': 700, 'Nf6': 738, 'Nf3': 686, 'Bg4': 691, 'e3': 736, 'e6': 685, 'Be2': 637, 'Bd6': 673, 'c4': 633, 'c6': 693, 'Nbd2': 711, 'Nbd7': 720, 'b4': 752, 'O-O': 713, 'Nc3': 637, 'Be7': 689, 'Qc2': 688, 'Re8': 705, 'e5': 618, 'e4': 707, 'Nce2': 667, 'c5': 696, 'Ng3': 662, 'Be6': 713, 'Bb5+': 702, 'Nd7': 668, 'd3': 705, 'f6': 729, 'b3': 674, 'Bb2': 715, 'Bf5': 693, 'f4': 725, 'h6': 637, 'Ne5': 671, 'h5': 707, 'g6': 686, 'Bg7': 698, 'Ne4': 664, 'Nc6': 699, 'd6': 675, 'Bxf3': 659, 'g3': 711, 'Bg2': 699, 'Ne2': 662, 'Nge7': 675, 'exd4': 675, 'Bb5': 650, 'Bd7': 732, 'a6': 660, 'Ba4': 747, 'Nd4': 682, 'Bc5': 722, 'Nxc6': 722, 'dxc6': 706, 'Ngf6': 709, 'b5': 663, 'a4': 709, 'dxc4': 690, 'Bxc4': 678, 'Nb6': 673, 'Qc7': 684, 'Qd6': 657, 'cxd5': 680, 'Nxd5': 698, 'b6': 648, 'Bb7': 710, 'h3': 692, 'c3': 705, 'Nd2': 692, 'Ngf3': 722, 'Bd3': 638, 'cxd4': 745, 'Qe2': 659, 'Bb4': 663, 'f5': 653, 'fxe4': 727, 'Bxe4': 726, 'Qh4': 682, 'Rb1': 675, 'Rb8': 684, 'cxb4': 651, 'axb4': 694

In [34]:
min( [v for (k,v) in dd.items()])

617

In [35]:
max( [v for (k,v) in dd.items()])

766

In [54]:
img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
dctx = ImageDraw.Draw(img)  # create drawing context
dh_top= 7
dh_bottom= 13
h= (img.height - dh_top - dh_bottom)/25.0
w= img.width/4.0

margin_left= w/4
margin_top= h/6

# def resize_fixed( img):
#     _dd = (271, 230) #width, height
#     return cv2.resize(img, _dd, interpolation = cv2.INTER_AREA) 

def generate_for( line):
#     if fnt is None:
#         fnt= random.choice( fonts)
#     print( fnt.getname())
    
    img = Image.open("C:/mestrado/datasets/syn/template/template.jpg")  # load base image
    dctx = ImageDraw.Draw(img)  # create drawing context

    i= 0
    for ln in range( 0, 8):
        for col in range( 0, 2):
            if i>= len( line):
                del dctx
                return img
        
            fnt= random.choice( fonts)
            txtsz = dctx.textsize(line[i], fnt)
        
            x= w* col + (w - txtsz[0])/2
            if random.random()< 0.7:
                xmargem= int( (w - txtsz[0])/3)
                x+= random.randint( -xmargem, xmargem)
            y= dh_top + h*ln + (h - txtsz[1])/2 #+ margin_top

            dctx.text( (x, y), line[i], font=fnt, fill="#000000")
            i= i+1
            
    del dctx 
    return img

def cut( img):
    h= 300
    w= 345

    img1= img[:h, :w]
    return img1

def generate_one( label): #pt_line, pgn, no, folder):
#     id_font= no%len( all_fonts)
    img= generate_for( label)
    img= cut( np.asarray(img))
    img= PIL.Image.fromarray(np.uint8(img))
    return img


# def __generate():
#     _files= label_files
# #     print( 'Gerando ', len( _files))
#     for i in range( 0, len( _files)):
#         if i%100== 0:
#             print( i)
#         img_file= _files[i].replace( 'labels', 'images.new').replace( '.pgn', '.jpg') 
#         img= generate_one( i, labelof( _files[i])[0:16], img_file)
        
#         img= random_transform( img, 0.3)
#         img.save( img_file)
# #         print( img_file)
# #         show( img__)
# #         show( transform( img__))
# #         return img
    
# __generate()
# img1= __generate()
# show( img1)
# pilimg= img1
# npimg= np.asarray( img1)


def __generate_random( N):
    dataset_dir= 'C:/mestrado/datasets/unified/random-32k-8lines--v6'
    
    N_train= int(N*0.8)
    print( 'N', N, 'N_train', N_train)
    
    for i in range( 0, N):
        if( i%100== 0):
            print( i)
        label_pt= random_label_from_vocab()
        img= generate_one( label_pt)
        img= random_transform( img, 0.3)
        pgn= to_pgn( label_pt)

        folder= 'train' if i< N_train else 'valid'
        
        # save
#         show( img)
        img_file= os.path.join( dataset_dir, folder, 'images', 'random-syn-'+ str(i+1).zfill( 5)+'.jpg')
        img.save( img_file)
        write_label( img_file.replace( 'images', 'labels').replace( '.jpg', '.pgn'), 
                    " ".join( to_pgn( label_pt)))

__generate_random( 30000)

N 30000 N_train 24000
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
1810

In [55]:
def gererate_all_labels():
    pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v6/*/labels/*.pgn')
    all_pgn= []
    for f in pgn_files:
        all_pgn.append( read_label( f))
        
    all_pgn= [' '.join( p.split()[0:16]) for p in all_pgn]
    
    write_label( 'C:/mestrado/datasets/unified/random-32k-8lines--v6/all-labels-.pgn', 
           '\n'.join( all_pgn))
    
gererate_all_labels()

In [7]:
pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v6/*/labels/*.pgn')




In [8]:
all_pgn= []
for f in pgn_files:
    all_pgn.append( read_label( f))

pat= "Nc3 d5 c4"

In [14]:
for i in range( 0, len( all_pgn)):
    if all_pgn[i].startswith( "Nc3 d5"):
        print( i, all_pgn[i])

2425 Nc3 d5 e4 c6 f4 g6 Nf3 Bg7 e5 h5 d4 Nh6 Be3 Bg4 h3 Nf5
2426 Nc3 d5 e4 d4 Nce2 e5 Ng3 Nf6 Nf3 Nc6 Bc4 Be7 O-O O-O d3 h6
2427 Nc3 d5 e4 e6 Nf3 Nf6 d3 Be7 Be2 c5 O-O Nc6 Re1 d4 Nb1 e5
2428 Nc3 d5 e4 e6 Nf3 Nf6 d3 Be7 Be2 c5 O-O Nc6 Re1 d4 Nb1 e5
2429 Nc3 d5 e4 e6 d3 c5 g3 Nf6 Bg2 Nc6 Bd2 Be7 Nf3 O-O O-O d4
27245 Nc3 d5 e4 d4 Nce2 e5 Ng3 Nf6 Bc4 Nc6 Nf3 Bd6 d3 h6 a3 Qe7
